In [2]:
import pandas as pd
import numpy as np


In [3]:
train_df = pd.read_csv("./data/train_E6oV3lV.csv",encoding="utf8")
test_df = pd.read_csv("./data/test_tweets_anuFYb8.csv",encoding="utf8")

In [4]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
# Others
import nltk
import string
import numpy as np
import pandas as pd

from sklearn.manifold import TSNE
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer


C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vivek.kumar5\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import re
stops = set(stopwords.words("english"))
stemmer = SnowballStemmer('english')


def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    #stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    text = text.split()
    #stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text

In [6]:
train_df['clean_tweet'] = train_df['tweet'].map(lambda x: clean_text(x))
test_df['clean_tweet'] = test_df['tweet'].map(lambda x: clean_text(x))

In [9]:
train_df.clean_tweet.iloc[3]

'model love take time ur ! ! !'

### OLD EDA

In [28]:
import re # regular expression
# https://pymotw.com/2/re/
import nltk
import string
from nltk.corpus import stopwords

stop = set(stopwords.words('english')) # set of stopwords in english
sno = nltk.stem.SnowballStemmer('english') # initializaing snowball stemmer

# this fn will replace an html tag with space
def cleanthtml(sentence): # fn to clean the word of any html tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ',sentence) # substitute
    return cleantext

def cleanpunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#]', r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/)]',r' ',cleaned)
    return cleaned

def clean_tweet(data):
    clean_list = []
    for tweet in data.values:
        tweet = cleanthtml(tweet)
        tweet = cleanpunc(tweet)
        sent = []
        for word in tweet.split(" "):
            if((word.isalpha()) & (len(word)>2)):
                if(word.lower() not in stop):
                    word = (sno.stem(word.lower()))
                    sent.append(word)
        clean_list.append(" ".join(sent))
    return clean_list

train_clean = clean_tweet(train_df.tweet)
test_clean = clean_tweet(test_df.tweet)
train_df['clean2'] = train_clean
test_df['clean2'] = test_clean

### Tokenize

In [161]:
# Getting positive words list
pos_words_list = []
neg_words_list = []
with open("./data/positive-words.txt") as fp:
    for line in fp:
        word = str(line.strip())
        pos_words_list.append(sno.stem(word))
        
with open("./data/negative-words.txt") as fp:
    for line in fp:
        word = str(line.strip())
        neg_words_list.append(sno.stem(word))      

In [162]:
#[sno.stem(x) for x in "hello achiever".split()]
print(train_df.clean_tweet[train_df.label == 1].iloc[10])
train_df.tweet[train_df.label == 1].iloc[10]

user user mock obama black user user user user brexit


'@user why not @user mocked obama for being black.  @user @user @user @user #brexit'

In [37]:
# splitting data
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
tweet_vect = TfidfVectorizer(ngram_range=(1,2))
tweet_vect.fit(train_df.clean_tweet)
train_tfidf = tweet_vect.transform(train_df.clean_tweet)
sub_tfidf = tweet_vect.transform(test_df.clean_tweet)

In [38]:
X_train, X_test, y_train, y_test = train_test_split(train_tfidf, train_df.label, test_size = 0.1)
X_train.shape

(28765, 190688)

### Tokenize 2

In [48]:
# splitting data

tweet_vect1 = TfidfVectorizer(ngram_range=(1,2))
tweet_vect.fit(train_df.clean2)
train_tfidf1 = tweet_vect.transform(train_df.clean2)
sub_tfidf1 = tweet_vect.transform(test_df.clean2)

X_train, X_test, y_train, y_test = train_test_split(train_tfidf1, train_df.label, test_size = 0.2)
X_train.shape

(25569, 165757)

## ML Models

In [68]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.metrics import f1_score
#rf = RandomForestClassifier(max_depth=10,n_estimators=100,n_jobs=-1,class_weight={0:1,1:5})
alpha = [0.001,0.005,0.01,0.05,0.1,0.5,1,21,55,100,200,300,400,450,501,510,520,550,600,700,900]
rf  = LogisticRegressionCV(Cs=alpha, class_weight="balanced", cv=3, penalty="l2",n_jobs=-1,solver="liblinear")
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
score = f1_score(y_test,y_pred)
print(score)

0.7158671586715866


In [ ]:
rf.predict(X_train)

In [177]:
num = 3
print(train_df.clean2[train_df.label==1].iloc[num])

lumpi say prove lumpi


#### little modification in train_df

In [237]:
# print(len(set(train_df.clean2[train_df.label==1].iloc[num].split()).intersection(set(pos_words_list))))
# print(len(set(train_df.clean2[train_df.label==1].iloc[num].split()).intersection(set(neg_words_list))))

positive_words = []
negative_words = []
for x,y in test_df.iterrows():
    p = len(set(y[3].split()).intersection(set(pos_words_list)))
    n = len(set(y[3].split()).intersection(set(neg_words_list)))
    positive_words.append(p)
    negative_words.append(n)


In [238]:
test_df['positive'] = positive_words
test_df['negative'] = negative_words
train_df.shape


(31962, 7)

,id,label,tweet,clean_tweet,clean2,positive,negative
0,1,0,@user when a father is dysfunctional and is s...,user father dysfunct selfish drag kid dysfunct...,father dysfunct selfish drag kid dysfunct run,0,2
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thank lyft credit ca not use caus of...,thank lyft credit cant use caus dont offer whe...,2,1
2,3,0,bihday your majesty,bihday majesti,bihday majesti,1,0
3,4,0,#model i love u take with u all the time in ...,model love take time ur ! ! !,model love take time,2,0
4,5,0,factsguide: society now #motivation,factsguid : societi motiv,societi motiv,1,0


In [345]:
filtered_data = train_df.drop_duplicates( subset={"clean2","label"},keep='first',inplace=False)


In [346]:
print(train_df.shape)
print(filtered_data.shape)

(31962, 7)
(27958, 7)


In [372]:

tweet_vect1 = TfidfVectorizer(ngram_range=(1,2))
tweet_vect1.fit(filtered_data.clean2)
train_tfidf1 = tweet_vect1.transform(filtered_data.clean2)
sub_tfidf1 = tweet_vect1.transform(test_df.clean2)

from scipy.sparse import hstack

train_data =hstack([train_tfidf1,filtered_data.positive.reshape(-1,1)])
train_data = hstack([train_data,filtered_data.negative.reshape(-1,1)])
test_data =hstack([sub_tfidf1,test_df.positive.reshape(-1,1)])
test_data = hstack([test_data,test_df.negative.reshape(-1,1)])
X_train1, X_test1, y_train1, y_test1 = train_test_split(train_data, filtered_data.label, test_size = 0.1)
print(train_data.shape)
print(train_tfidf1.shape)

(27958, 165759)
(27958, 165757)


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead



In [365]:
# without filtered for log reg
tweet_vect_orig_data = TfidfVectorizer(ngram_range=(1,2))
tweet_vect_orig_data.fit(train_df.clean2)
train_tfidf_orig = tweet_vect1.transform(train_df.clean2)
sub_tfidf_orig = tweet_vect1.transform(test_df.clean2)

from scipy.sparse import hstack

train_data_orig =hstack([train_tfidf_orig,train_df.positive.reshape(-1,1)])
train_data_orig = hstack([train_data_orig,train_df.negative.reshape(-1,1)])
test_data_orig = hstack([sub_tfidf_orig,test_df.positive.reshape(-1,1)])
test_data_orig = hstack([test_data_orig,test_df.negative.reshape(-1,1)])
X_train, X_test, y_train, y_test = train_test_split(train_data_orig, train_df.label, test_size = 0.1)
print(train_data_orig.shape)
print(test_data_orig.shape)

(31962, 165759)
(17197, 165759)


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning:

reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead



In [382]:
log_reg_cv  = LogisticRegressionCV(Cs=alpha, class_weight="balanced", cv=3, penalty="l2",n_jobs=-1,solver="liblinear")
log_reg_cv.fit(X_train,y_train)
print(f1_score(y_test, log_reg_cv.predict(X_test))*100)
# sub_pred = log_reg_cv.predict(test_data_orig)
# test_df['label'] = sub_pred
# sub_df = test_df[['id','label']]
# sub_df.to_csv("./sub/LogRegPosNeg2.csv",index=False)
# print(f1_score(y_test,sub_pred))

72.53333333333333


## DT

In [419]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

for i in [1,2,3,4,5,6]:
    rf = RandomForestClassifier(n_estimators=i,max_depth=100)
    #dt = DecisionTreeClassifier(max_depth=i)
    rf.fit(X_train,y_train)
    print(f1_score(y_test, rf.predict(X_test))*100)

32.97491039426523
42.76315789473684
26.229508196721312
34.883720930232556
29.1497975708502
32.0


In [433]:
from sklearn.svm import SVC
svm = SVC(gamma=2,C=5,kernel="rbf")
svm.fit(X_train,y_train)
print(f1_score(y_test, svm.predict(X_test))*100)

40.90909090909091


In [ ]:
gamm_list = [0.01,1,10]
C_list = [400,700,900,1200]

for i in gamm_list:
    #for j in C_list:
    svm = SVC(gamma=i,kernel="rbf")
    svm.fit(X_train,y_train)
    print("gama:",i," C:",j,f1_score(y_test, svm.predict(X_test))*100)


gama: 0.01  C: 400 6.451612903225806
gama: 1  C: 400 40.90909090909091


In [276]:
log_reg_score = f1_score(y_test1,rf.predict(X_test1))
print(log_reg_score)
# model-2
randforest = RandomForestClassifier(class_weight={0:1,1:3})
randforest.fit(X_train1,y_train1)
y_pred = randforest.predict(X_test1)
score = f1_score(y_test1,y_pred)
print(score)

0.5398773006134969


In [81]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)
score = f1_score(y_test,y_pred)
print(score)

KeyboardInterrupt: 

In [82]:
sub_pred = dt.predict(sub_tfidf1)
test_df['label'] = sub_pred
sub_df = test_df[['id','label']]
sub_df.to_csv("./sub/DT.csv",index=False)

AttributeError: 'list' object has no attribute 'take'

TypeError: A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.